<a href="https://colab.research.google.com/github/brunogon17/imersao_ia/blob/main/Projeto_Imersao_IA_Google_e_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [20]:
#Configurações iniciais
import google.generativeai as genai
from google.colab import userdata

minha_api_key = userdata.get('minha_api_key')
GOOGLE_API_KEY=minha_api_key
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)



In [93]:
import json #importo a biblioteca para trabalhar com JSON
import random #importo a biblioteca para utilizar o random
valor_consulta = random.randint(0, 3) # crio um random com valores ( 1,2,3 ou 4)

#Crio uma tabela de exemplo para simular um banco de dados com placa e se pesou ou nao
tabela_sistema_pesagem =[
    {
    'id': 1,
    'placa': 'ABC1234',
    'pesou': 'Sim'
    },
    {
    'id': 2,
    'placa': 'AQR1212',
    'pesou': 'NA'
    },
    {
    'id': 3,
    'placa': 'QQQ1111',
    'pesou': 'Nao'
    },
    {
    'id': 4,
    'placa': 'WQW9999',
    'pesou': 'Sim'
    }
]

#Trato os dados da tabela de exemplo acima
dados = tabela_sistema_pesagem[valor_consulta]
foto = 'www/deteccoes/' + str(dados['id']) + ".png" # pego o id e acrescendo .png para ficar igual o nome das fotos na pasta www
placa = dados['placa'] # pego a placa
pesou = dados['pesou'] # pego se pesou ou nao

#Crio agora uma lista com fotos para passar para o modelo dizendo o que é carreta e ou que é carro!
prompt_parts = [
    "Object: ",
     genai.upload_file('www/carreta.png'),
    "Description: caminhão",
    "Object: ",
     genai.upload_file('www/carreta1.png'),
    "Description: caminhão",
    "Object: ",
     genai.upload_file('www/carreta.png'),
    "Description: caminhão",
    "Object: ",
     genai.upload_file('www/outro.png'),
    "Description: carro",
     genai.upload_file(foto),
    "Description: ",
     ]

#peço o modelo para verificar o tipo de veiculo no momento
response = model.generate_content(prompt_parts)

tipo_veiculo = response.text
print("")
print("Tipo do veiculo: ",tipo_veiculo)
print("")
print("*************************************************")
print("")
print("")
if(tipo_veiculo == "caminhão"):
  #Verifico se esta cheia ou vazia
  #exemplos para o gemini entende o que é cheio e o que é vazio
  prompt_parts2 = [
      "Object: ",
      genai.upload_file('www/cheia.png'),
      "Description: cheia",
      "Object: ",
      genai.upload_file('www/vazia.png'),
      "Description: vazia",
      "Object: ",
      genai.upload_file(foto),
      "Description: ",
      ]
  #peço o modelo para verificar o tipo de veiculo no momento
  response2 = model.generate_content(prompt_parts2)
  condicao_do_veiculo = response2.text
  if(condicao_do_veiculo == "cheia"):
    #Se trata de um caminhão saindo cheio, entao preciso verificar se pesou, caso contrario está desviando uma carga!
    if(pesou == "Sim"):
      print("Se trata de um caminhão que está saindo cheio e que pesou na balança emitindo sua nota fiscal para transporte. Esta tudo ok com o veiculo placa " + placa)
    else:
      print("Se trata de um caminhão saindo cheio e que não pesou, devo acionar a equipe de segurança patrimonial pois o veiculo placa " + placa + " possivelmente está desviando uma carga e saindo sem nota fiscal emitida!")
  else:
    #Se trata de um caminhão, porem vazio, entao permito ir embora sem ao menos verificar se pesou emitindo a nota fical para transporte
    print("Se trata de um caminhão placa " + placa + " e está saindo vazio!")


elif (tipo_veiculo == "carro"):
  print("Se trata de um carro, logo, permito sair!")


else:
 print("Ignora pois houve algum erro de detecção!")



Tipo do veiculo:  caminhão

*************************************************


Se trata de um caminhão que está saindo cheio e que pesou na balança emitindo sua nota fiscal para transporte. Esta tudo ok com o veiculo placa WQW9999
